In [27]:
import pandas as pd

## Institutes

In [28]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII179988,Reggio Calabria Arghillà,Casa circondariale,294,1,326,135,164,12,32,...,85.0,0.0,3.0,85.0,85.0,85.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
1,MII172610,Brescia Verziano,Casa di reclusione,71,1,115,77,95,2,0,...,71.0,0.0,0.0,71.0,10.0,71.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
2,MII172320,Busto Arsizio,Casa circondariale,240,14,438,184,190,15,21,...,154.0,170.0,0.0,177.0,177.0,176.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
3,MII173712,Como,Casa circondariale,226,1,430,198,216,17,23,...,55.0,177.0,0.0,200.0,4.0,4.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
4,MII173747,Cremona,Casa circondariale,394,10,599,177,202,17,23,...,68.0,216.0,4.0,252.0,77.0,77.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20


## Most recent data

In [29]:
# Get the most recent update
df['posti_aggiornati_al'] = pd.to_datetime(df['posti_aggiornati_al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id')['posti_aggiornati_al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti_reali'] = df_most_recent['posti_regolamentari'] - df_most_recent['posti_non_disponibili']
df_most_recent['tasso_sovraffollamento'] = round(((df_most_recent['posti_occupati'] / df_most_recent['posti_reali'])*100),0)

KeyError: 'posti_aggiornati_al'

In [ ]:
df_most_recent.value_counts(['posti_aggiornati_al'])

In [ ]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id'})

merged_df = pd.merge(df_most_recent, df_info, on='id')

# Adding additional columns
merged_df['posti_reali'] = merged_df['posti_regolamentari'] - merged_df['posti_non_disponibili']

merged_df['tasso_affollamento'] = round((merged_df['posti_occupati'] / merged_df['posti_reali']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id',
    'nome',
    'tasso_affollamento',
    'indirizzo',
    'tipo',
   'posti_regolamentari',
    'posti_non_disponibili',
    'posti_reali',
    'posti_occupati',
    'posti_aggiornati_al',
    'personale_polizia_previsti',
    'personale_polizia_effettivi',
    'personale_polizia_aggiornato_a',
    'personale_amministrativi_previsti',
    'personale_amministrativi_effettivi',
    'personale_amministrativo_aggiornato_al',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head()

In [ ]:
df_filtered.head()

In [ ]:
df_filtered['scheda_istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id'] + '">Vai alla scheda istituto.</a>'


df_filtered.head()

In [8]:
# df_filtered['personale_polizia_mancanti'] = df_filtered['personale_polizia_previsti'] - df_filtered['personale_polizia_effettivi']
# df_filtered['personale_polizia_mancanti_perc'] = round(df_filtered['personale_polizia_mancanti'] / df_filtered['personale_polizia_previsti'] * 100, 2)

# df_filtered.sort_values('personale_polizia_mancanti_perc', ascending=False)

In [9]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [ ]:
grouped_df = df.groupby('posti_aggiornati_al').sum(numeric_only=True).reset_index()
grouped_df

In [ ]:
grouped_df['posti_reali'] = (grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']).round(0)
grouped_df['tasso_sovraffollamento'] = ((grouped_df['posti_occupati'] / grouped_df['posti_reali']))*100

grouped_df.head()


In [ ]:
grouped_df = grouped_df[['posti_aggiornati_al', 'posti_regolamentari', 'posti_non_disponibili', 'posti_occupati']]
grouped_df['posti_disponibili'] = grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']
grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


In [13]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [ ]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome', 'posti_occupati', 'tasso_affollamento', 'personale_polizia_previsti', 'personale_polizia_effettivi', 'personale_polizia_aggiornato_a', 'posti_aggiornati_al']]

df_polizia['personale_polizia_mancanti'] = df_polizia['personale_polizia_previsti'] - df_polizia['personale_polizia_effettivi']

df_polizia.head(2)

In [ ]:
df_polizia['personale_polizia_mancanti_perceent'] = round(df_polizia['personale_polizia_mancanti'] / df_polizia['personale_polizia_previsti']*100,2)
df_polizia.head(2)


In [ ]:
df_polizia_clean = df_polizia[df_polizia['posti_occupati'] != 0]
df_polizia_clean.sort_values('personale_polizia_mancanti_perceent', ascending=False).head(2)

In [ ]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso_affollamento'] > 120) & (df_polizia_clean['personale_polizia_mancanti_perceent'] > 20)].reset_index(drop=True)
critical_prisons

In [18]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [19]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [20]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'posti_aggiornati_al': 'Date', 'tasso_affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [21]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
df2.tail(2)

In [ ]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

In [ ]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


In [ ]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


In [26]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)